# Orthocorrection of KH-5 ARGON images

The purpose of this notebook is to ortho correct Declassified Intelligence Satellite Photography (DISP) from the KH-5 ARGON missions. The images are loaded one by one and corrected using the functions provided in geometry.

In [1]:
import os
import posixpath
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt

# personnal packages
import utils.plots as plots
import geometry.internal_orientation as io

# to keep xarray quiet
# import warnings
# warnings.filterwarnings("ignore")

%matplotlib qt

In [2]:
data_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/KH-5_ARGON_images'
products = []

for x in os.listdir(data_root):
    if os.path.isdir(posixpath.join(data_root, x)):
        products.append(x)
        
products

['DS09034A007MC018', 'DS09058A024MC013']

# DS09034A007MC018

In [3]:
file = posixpath.join(data_root, "DS09034A007MC018", "DS09034A007MC018_a.tif")
raster = rioxarray.open_rasterio(file)
image = raster.to_numpy()
image = image[0, :, :]
raster

c:\ProgramData\miniconda3\Lib\site-packages\rioxarray\_io.py:1143: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore


<xarray.DataArray (band: 1, y: 19928, x: 19363)> Size: 386MB
array([[[0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0],
        ...,
        [0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0]]], dtype=uint8)
Coordinates:
  * band         (band) int32 4B 1
  * x            (x) float64 155kB 0.5 1.5 2.5 ... 1.936e+04 1.936e+04 1.936e+04
  * y            (y) float64 159kB 0.5 1.5 2.5 ... 1.993e+04 1.993e+04 1.993e+04
    spatial_ref  int32 4B 0
Attributes:
    TIFFTAG_SOFTWARE:        Adobe Photoshop CC 2018 (Windows)
    TIFFTAG_DATETIME:        2019:12:09 13:21:47
    TIFFTAG_XRESOLUTION:     1434.0083
    TIFFTAG_YRESOLUTION:     1434.0083
    TIFFTAG_RESOLUTIONUNIT:  3 (pixels/cm)
    scale_factor:            1.0
    add_offset:              0.0

## IO

### Image coordinates
First, the IO parameters are assessed. The image coordinates of the fiducial markers (FMs) must first be writen down .
Since the full image is too heavy to be entirely displayed by matplotlib, tiles are displayed successively using the next cell.

In [4]:
shx, shy = image.shape
tiles = ["00", "01", "02", "03", "04", "14", "24", "34", "44", "43", "42", "41", "40", "30", "20", "10"]
# offset_x, offset_y = plots.plot_image(image, fraction=5, tile=tiles[0], rescale=True) 

In [5]:
# [814 + offset_x, 3294 + offset_y]

In [6]:
FMs_image_coords = np.array([
    # left side
    [814, 3294],
    [836, 6198],
    [864, 9082],
    [859, 11982],
    [843, 14862],
    # bottom side
    [3765, 17761],
    [6637, 17737],
    [9547, 17728],
    [12447, 17749],
    [15319, 17712],
    # right side
    [18202, 14792],
    [18216, 11938],
    [18188, 9033],
    [18184, 6130],
    [18172, 3236],
    # top side
    [15254, 353],
    [12380, 370],
    [9497, 401],
    [6621, 374],
    [3720, 414]
])

### Fiducial coordinates
Without the calibration data, it is only possible to guess the fiducial coordinates of the FMs. This is done by assuming they are aligned, and by averaging their coordinates.

In [7]:
# # y location of top and bottom lines
# print(127/shy * (FMs_image_coords[15:20, :].mean(axis=0)[1]), 127/shy * (FMs_image_coords[5:10, :].mean(axis=0)[1]))

# # x location of left and right lines
# print(127/shx * (FMs_image_coords[0:5, :].mean(axis=0)[0]), 127/shx * (FMs_image_coords[10:15, :].mean(axis=0)[0]))

# # y locations of markers on vertical lines
# print("======")
# print(127/shy * (FMs_image_coords[[0, 14], :].mean(axis=0)[1]))
# print(127/shy * (FMs_image_coords[[1, 13], :].mean(axis=0)[1]))
# print(127/shy * (FMs_image_coords[[2, 12], :].mean(axis=0)[1]))
# print(127/shy * (FMs_image_coords[[3, 11], :].mean(axis=0)[1]))
# print(127/shy * (FMs_image_coords[[4, 10], :].mean(axis=0)[1]))

# # x locations of markers on horizontal lines
# print("======")
# print(127/shx * (FMs_image_coords[[5, 19], :].mean(axis=0)[0]))
# print(127/shx * (FMs_image_coords[[6, 18], :].mean(axis=0)[0]))
# print(127/shx * (FMs_image_coords[[7, 17], :].mean(axis=0)[0]))
# print(127/shx * (FMs_image_coords[[8, 16], :].mean(axis=0)[0]))
# print(127/shx * (FMs_image_coords[[9, 15], :].mean(axis=0)[0]))

In [8]:
FMs_fiducial_coords = np.array([
    # left side
    [5.37, 21.41],
    [5.37, 40.42],
    [5.37, 59.40],
    [5.37, 78.44],
    [5.37, 97.24],
    # bottom side
    [23.85, 116.33],
    [42.24, 116.33],
    [60.68, 116.33],
    [79.11, 116.33],
    [97.41, 116.33],
    # right side
    [115.93, 97.24],
    [115.93, 78.44],
    [115.93, 59.40],
    [115.93, 40.42],
    [115.93, 21.41],
    # top side
    [97.41, 2.50],
    [79.11, 2.50],
    [60.68, 2.50],
    [42.24, 2.50],
    [23.85, 2.50]
])

In [9]:
fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
axs[0].plot(FMs_fiducial_coords[:, 0], FMs_fiducial_coords[:, 1], linestyle="", marker="o", color="r", label='FMs fiducial coords')
# axs[0].grid()
axs[0].set_title("Fiducial coordinates [mm]")
axs[1].plot(FMs_image_coords[:, 0], FMs_image_coords[:, 1], linestyle="", marker="o", color="b", label='FMs image coords')
# axs[1].grid()
axs[1].set_title("Image coordinates [pixels]")

Text(0.5, 1.0, 'Image coordinates [pixels]')

### Transformation parameters
Finally, the image to fiducial coordinates transformation parameters are retrieved via an optimization process.
Several transformations and optimization methods are tested:

1. the affine transformation described in Molnar et al. (2021):
$$
\begin{pmatrix}
\xi\\
\eta
\end{pmatrix} = \begin{pmatrix}
\delta\xi\cos\alpha & \delta\eta\sin\alpha\\
-\delta\xi\sin\alpha & \delta\eta\cos\alpha
\end{pmatrix} \begin{pmatrix}
x - x_c\\
y - y_c
\end{pmatrix}
$$


where:
- $\xi$ and $\eta$ are the fiducial coordinates
- $x$ and $y$ are the image coordinates
- $\alpha$ is an angle
- $\delta\xi$ and $\delta\eta$ are the pixel size
- $x_c$ and $y_c$ are the location of the image coordinates system center.

The transformation is linear but the relation with the parameters is not (especially $\alpha$). Therefore, several optimization techniques are tested; the only satisfactory one is the Nelder-Mead algorithm (simplex).


In [10]:
res = opt.minimize(
    io.objective_function,
    x0=[0, 0, 0, 127/shx, 127/shy],
    args=(FMs_fiducial_coords, FMs_image_coords),
    method='Nelder-Mead', # I is not clear why it works with this method ans not with BFGS
)
if res.success == False:
    print(res)
params = res.x

xc, yc, alpha, delta_eta, delta_xi = params[0], params[1], params[2], params[3], params[4]
print(xc, yc, alpha, delta_eta, delta_xi)

FMs_inferred_fiducial_coords = np.array([
    io.image_to_fiducial_coordinates(FMs_image_coords[i, 0], FMs_image_coords[i, 1], xc, yc, alpha, delta_eta, delta_xi) for i in range(FMs_image_coords.shape[0])
])

if res.success:
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(FMs_fiducial_coords[:, 0], FMs_fiducial_coords[:, 1], linestyle="", marker="x", color="b", label='FMs fiducial coords')
    ax.plot(FMs_inferred_fiducial_coords[:, 0], FMs_fiducial_coords[:, 1], linestyle="", marker="+", color="r", label='FMs inferred fiducial coords')
    ax.set_title("Fiducial coordinates [mm] (non linear optimization)")

0.0008993471699334189 0.000887007905505168 -0.0018796299388844332 0.00638032704399426 0.00655039327757314


2. A linear transformation of the plane:
$$
\begin{pmatrix}
\xi\\
\eta
\end{pmatrix} = \begin{pmatrix}
m_{11} & m_{12} \\
m_{21}  & m_{22} 
\end{pmatrix} \begin{pmatrix}
x\\
y
\end{pmatrix}
$$

Where:
- $\xi$ and $\eta$ are the fiducial coordinates
- $x$ and $y$ are the image coordinates
- $m_{11}\cdots m_{22}$ are a translation-rotation matrix coefficients.

This time, the relation between the parameters is fully linear, and allows for a linear least squares regression. Unfortunately, this method does not give good results (probably due to ill-conditioning), even with a Tikhonov regularization. Iterative methods, such as BFGS or Nelder-Mead are satisfactory.  

In [24]:
matrix = io.least_squares_linear(FMs_fiducial_coords, FMs_image_coords, alpha=1e11, a_priori=np.array([[127/shx, 0], [0, 127/shy]]))
print(matrix)

FMs_inferred_fiducial_coords = np.array([
    io.image_to_fiducial_coordinates_linear(FMs_image_coords[i, 0], FMs_image_coords[i, 1], matrix) for i in range(FMs_image_coords.shape[0])
])

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(FMs_fiducial_coords[:, 0], FMs_fiducial_coords[:, 1], linestyle="", marker="x", color="b", label='FMs fiducial coords')
ax.plot(FMs_inferred_fiducial_coords[:, 0], FMs_fiducial_coords[:, 1], linestyle="", marker="+", color="r", label='FMs inferred fiducial coords')
ax.set_title("Fiducial coordinates [mm] (linear least squares)")

[[ 6.65108140e-03 -5.05776447e-06]
 [-4.82764561e-06  6.84564463e-03]]


Text(0.5, 1.0, 'Fiducial coordinates [mm] (linear least squares)')

In [35]:
res = opt.minimize(
    io.objective_function_linear,
    x0=[127/shx, 0, 0, 127/shy],
    args=(FMs_fiducial_coords, FMs_image_coords),
    method='BFGS'
)
if res.success == False:
    print(res)
matrix = res.x
matrix = np.array([
    [matrix[0], matrix[1]],
    [matrix[2], matrix[3]]
])
print(matrix)

FMs_inferred_fiducial_coords = np.array([
    io.image_to_fiducial_coordinates_linear(FMs_image_coords[i, 0], FMs_image_coords[i, 1], matrix) for i in range(FMs_image_coords.shape[0])
])

if res.success:
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(FMs_fiducial_coords[:, 0], FMs_fiducial_coords[:, 1], linestyle="", marker="x", color="b", label='FMs fiducial coords')
    ax.plot(FMs_inferred_fiducial_coords[:, 0], FMs_fiducial_coords[:, 1], linestyle="", marker="+", color="r", label='FMs inferred fiducial coords')
    ax.set_title("Fiducial coordinates [mm] (linear optimization)")

[[ 6.37959723e-03 -1.13521386e-05]
 [ 1.27302559e-05  6.54976449e-03]]


In [34]:
res = opt.minimize(
    io.objective_function_linear,
    x0=[127/shx, 0, 0, 127/shy],
    args=(FMs_fiducial_coords, FMs_image_coords),
    method='Nelder-Mead'
)
if res.success == False:
    print(res)
matrix = res.x
matrix = np.array([
    [matrix[0], matrix[1]],
    [matrix[2], matrix[3]]
])
print(matrix)

FMs_inferred_fiducial_coords = np.array([
    io.image_to_fiducial_coordinates_linear(FMs_image_coords[i, 0], FMs_image_coords[i, 1], matrix) for i in range(FMs_image_coords.shape[0])
])

if res.success:
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(FMs_fiducial_coords[:, 0], FMs_fiducial_coords[:, 1], linestyle="", marker="x", color="b", label='FMs fiducial coords')
    ax.plot(FMs_inferred_fiducial_coords[:, 0], FMs_fiducial_coords[:, 1], linestyle="", marker="+", color="r", label='FMs inferred fiducial coords')
    ax.set_title("Fiducial coordinates [mm] (linear optimization)")

[[ 6.37953922e-03 -1.11184367e-05]
 [ 1.25295294e-05  6.54993851e-03]]


Conclusion:
The impact of these methods on the quality of the georeferencing need to be assessed. For now, the parameters of the linear-transformation obtained with the Nelder-Mead algorithm are kept.

## EO